In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [2]:
!nvidia-smi # test we are on GPU

Wed Nov 19 22:39:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
%pip uninstall -y torch torchaudio torchvision

%pip install torch==2.9.1 --index-url https://download.pytorch.org/whl/test/cu126 -q # 2.9.1 has Muon
%pip install torchaudio==2.9.1 --index-url https://download.pytorch.org/whl/test/cu126 -q 

%pip install torchvision==0.24.1+cu126 --index-url https://download.pytorch.org/whl/cu126 -q

%pip install timm wandb==0.22.0 torchmetrics numpy tensorboard matplotlib -q #--no-deps -q # no-deps to avoid torch 2.8.0, but wandb needs newer vers



Found existing installation: torch 2.9.1+cu126
Uninstalling torch-2.9.1+cu126:
  Successfully uninstalled torch-2.9.1+cu126
Found existing installation: torchaudio 2.9.1+cu126
Uninstalling torchaudio-2.9.1+cu126:
  Successfully uninstalled torchaudio-2.9.1+cu126
Found existing installation: torchvision 0.24.1+cu126
Uninstalling torchvision-0.24.1+cu126:
  Successfully uninstalled torchvision-0.24.1+cu126
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.8.5 requires torchvision>=0.11, which is not installed.
timm 1.0.22 requires torchvision, which is not installed.


In [4]:
import os

import torch
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from torchvision.transforms import v2
from torch.backends import cudnn
from torch import GradScaler
from torch import optim
from tqdm import tqdm
import numpy as np
import pickle
import time

from torchvision.datasets import CIFAR10, CIFAR100, MNIST, OxfordIIITPet
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import timm
import wandb
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter



In [5]:
print("Muon" in dir(torch.optim))
print(torch.__version__)

True
2.9.1+cu126


In [6]:
device = torch.accelerator.current_accelerator() if torch.accelerator.is_available() else torch.device("cpu")
enable_half = device.type != "cpu"
scaler = GradScaler(device, enabled=enable_half)

print("Grad scaler is enabled:", enable_half)
print("Device:", device)

Grad scaler is enabled: True
Device: cuda


In [7]:
config = {
    "batch_size": 64,
    "num_workers": 2,
    "pin_memory": True,
    "epochs": 50,
    "lr": 1e-1,
    "model": "resnet18",
    "image_size": 224,
    "use_amp": True
}

In [8]:
basic_transforms = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
])

train_transforms = v2.Compose([
    v2.ToImage(),
    v2.RandomCrop(32, padding=4),
    v2.RandomHorizontalFlip(),
    v2.ToDtype(torch.float32, scale=True),
    # v2.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])

test_transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    # v2.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])

train_dataset= CIFAR100(root='./data', train=True, download=True, transform=train_transforms)
test_dataset = CIFAR100(root='./data', train=False, download=True, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True, num_workers=config["num_workers"], pin_memory=config["pin_memory"])
test_loader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False, num_workers=config["num_workers"], pin_memory=config["pin_memory"])

In [9]:
model = timm.create_model(config["model"], pretrained=False, num_classes=100)

model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = nn.Identity()

model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): Identity()
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act1): ReLU(inplace=True)
      (aa): Identity()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_

In [10]:
def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    return train_loss / len(train_loader), 100.0 * correct/total


def test(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    return test_loss / len(test_loader), 100.0 * correct/total

def train_model(train_loader, test_loader, model, epochs=config["epochs"], batch_size=config["batch_size"], lr=config["lr"]):
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
    scheduler = StepLR(optimizer, step_size=30, gamma=0.1)
    criterion = nn.CrossEntropyLoss()

    writer = SummaryWriter(log_dir=f'./logs/cifar100')

    best_acc = 0.0

    best_loss = None
    counter = 0
    patience = 10


    start_time = time.time()
    print("Start training")
    
    for epoch in range(epochs):
        epoch_start = time.time()

        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        test_loss, test_acc = test(model, test_loader, criterion, device)

        scheduler.step()
        current_lr = optimizer.param_groups[0]['lr']
        epoch_time = time.time() - epoch_start

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%, LR: {current_lr:.6f}, Time: {epoch_time:.2f}s")

        writer.add_scalar('Loss/Train', train_loss, epoch)
        writer.add_scalar('Loss/Test', test_loss, epoch)
        writer.add_scalar('Accuracy/Train', train_acc, epoch)
        writer.add_scalar('Accuracy/Test', test_acc, epoch)
        writer.add_scalar('Learning Rate', current_lr, epoch)

        if test_acc > best_acc:
            best_acc = test_acc
            os.makedirs('./checkpoints', exist_ok=True)

            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_acc': best_acc,        
            }, f'./checkpoints/cifar_best_model.pth')
            print(f"Best model saved with accuracy: {best_acc:.2f}%")

        if best_loss is None:
            best_loss = test_loss
        elif test_loss > best_loss:
            counter += 1
            if counter >= patience:
                print(f"Early stopping triggered for epoch {epoch+1}")
                break
        else:
            best_loss = test_loss
            counter = 0

    total_time = time.time() - start_time
    print(f"Training completed in {total_time/60:.2f} minutes")
    print(f"Best Test Accuracy: {best_acc:.2f}%")
    
    writer.close()

    return {
        'best_acc': best_acc,
        'total_time': total_time,
        'final_train_acc': train_acc,
        'final_test_acc': test_acc
    }


In [ ]:
results = train_model(train_loader, test_loader, model, config["epochs"], config["batch_size"], config["lr"])


Start training
Epoch 1/50, Train Loss: 3.8425, Train Acc: 10.91%, Test Loss: 3.4793, Test Acc: 16.24%, LR: 0.100000, Time: 43.19s
Best model saved with accuracy: 16.24%
Epoch 2/50, Train Loss: 3.1362, Train Acc: 22.36%, Test Loss: 3.0392, Test Acc: 24.33%, LR: 0.100000, Time: 44.53s
Best model saved with accuracy: 24.33%
Epoch 3/50, Train Loss: 2.5881, Train Acc: 32.67%, Test Loss: 2.4548, Test Acc: 35.74%, LR: 0.100000, Time: 44.84s
Best model saved with accuracy: 35.74%
Epoch 4/50, Train Loss: 2.2893, Train Acc: 39.08%, Test Loss: 2.5421, Test Acc: 34.62%, LR: 0.100000, Time: 45.54s
Epoch 5/50, Train Loss: 2.1054, Train Acc: 42.89%, Test Loss: 2.2748, Test Acc: 40.59%, LR: 0.100000, Time: 44.62s
Best model saved with accuracy: 40.59%
Epoch 6/50, Train Loss: 1.9659, Train Acc: 46.09%, Test Loss: 2.1633, Test Acc: 43.25%, LR: 0.100000, Time: 44.77s
Best model saved with accuracy: 43.25%
Epoch 7/50, Train Loss: 1.8708, Train Acc: 48.60%, Test Loss: 1.9918, Test Acc: 46.57%, LR: 0.100000